## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [1]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam

Using TensorFlow backend.


In [2]:
batch_size = 128 
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 50 # 訓練的 epochs 數量

# 讀取資料並檢視

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:
data_generator = ImageDataGenerator(rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='wrap')

In [4]:
from keras.callbacks import EarlyStopping

earlystop = EarlyStopping(monitor="val_acc", patience=10, verbose=1 )

In [5]:
from keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(factor=0.5, 
                              min_lr=1e-12, 
                              monitor='val_loss', 
                              patience=5, 
                              verbose=1)

In [6]:
# 建立 ResNet 模型
from model_builder import CNNmodel # 這是從 resnet_builder.py 中直接 import 撰寫好的 resnet 函數
model = CNNmodel(input_shape=(32,32,3)) 
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 30, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 48)        13872     
_________________________________________________________________
batch_normalization_2 (Batch (None, 28, 28, 48)        192       
_________________________________________________________________
activation_2 (Activation)    (None, 28, 28, 48)        0         
__________

In [7]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=1e-3),
              metrics=['accuracy'])
#使用分批資料訓練模型

model.fit_generator(data_generator.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=int(len(x_train) /32),
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test,y_test),
                    
                   callbacks=[earlystop,reduce_lr])
    
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/50
1562/1562 [==============================] - 64s 41ms/step - loss: 1.5028 - acc: 0.4453 - val_loss: 1.2866 - val_acc: 0.5579
Epoch 2/50
1562/1562 [==============================] - 58s 37ms/step - loss: 1.1095 - acc: 0.6061 - val_loss: 0.9976 - val_acc: 0.6483
Epoch 3/50
1562/1562 [==============================] - 59s 38ms/step - loss: 0.9535 - acc: 0.6669 - val_loss: 1.1456 - val_acc: 0.6305
Epoch 4/50
1562/1562 [==============================] - 59s 38ms/step - loss: 0.8577 - acc: 0.7029 - val_loss: 1.1245 - val_acc: 0.6442
Epoch 5/50
1562/1562 [==============================] - 59s 38ms/step - loss: 0.7967 - acc: 0.7263 - val_loss: 0.6581 - val_acc: 0.7685
Epoch 6/50
1562/1562 [==============================] - 59s 38ms/step - loss: 0.7466 - acc: 0.7426 - val_loss: 0.6970 - val_acc: 0.7609
Epoch 7/50
1562/1562 [==============================] - 59s 38ms/step - loss: 0.7083 - acc: 0.7565 - val_loss: 0.8468 - val_acc: 0.7247
Epoch 8/50
1562/1562 [==========================